## Tutorial: Create an Isolated VLAN and Servers

This notebook demonstrates how to create isolated VLANs and attach servers.

#### Modules 

- [Reserve Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)
- [Get Network by Name](../modules-python/network/get_network_by_name.ipynb)

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [ ]:
import json
import os
import chi

from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *

#Config with your project and site
chi.set('project_name', 'CH-816532') # Replace with your project name
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

#Insert keypair name
key_name = 'pruth-jupyter'  # Change to your keypair

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Server attributes
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_haswell"
server_count=4

#### Create a Lease

This cell creates a multi-reservation lease with one server, one private network, and one floating IP address. The start date is set to one miniute from now and the end date is set to one day from now.  Then a list of reservations is created by calling one or more chi example funtions as described in this [module](../modules-python/reservations/reserve_multiple_resources.ipynb).  The name, dates, and list of reservations are passed to the OpenStack Blazar API to create the lease. 

After creating the lease you can view the lease information by [getting the lease](../modules-python/reservations/get_lease_by_name.ipynb) and printing the json object that is returned.

In [ ]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
lease = get_lease(lease_name)
    
#Print the lease info
print(json.dumps(lease, indent=2))

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
#Get the lease by name
lease = get_lease(lease_name)

compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', lease['reservations']))[0]['id']
network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', lease['reservations']))[0]['id']
floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', lease['reservations']))[0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("network_reservation_id: " + network_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [ ]:
network = get_network_by_name(network_name)
network_id = network['id']

print('Network ID: ' + str(network_id))

#### Add a subnet

[Adds a subnet](../modules-python/network/create_subnet.ipynb) to the reserved network. 

In [ ]:
subnet = add_subnet(subnet_name, network_name)

#### Add a Router

TODO: add links here

In [ ]:
router = create_router(router_name, network_name)

#### Attach the Router and Subnet

TODO: Add links here

In [ ]:
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

#### Start the Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [ ]:
#create the server
server = create_server(server_name, 
                       reservation_id=compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)

#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [ ]:
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

#### Delete Server

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
delete_server_by_name(server_name)

#### De-configure Network
TODO: break up into steps

In [ ]:
try:
    result = detach_router_by_name(router_name=router_name, subnet_name=subnet_name)
except Exception as e:
    print("detach_router_by_name error:" + str(e))
    pass

try:
    result = delete_router_by_name(router_name)
except Exception as e:
    print("delete_router_by_name error: " + str(e))
    pass

try:
    result = delete_subnet_by_name(subnet_name)
except Exception as e:
    print("delete_subnet_by_name error: " + str(e))
    pass

try:
    result = delete_network_by_name(network_name)
except Exception as e:
    print("delete_network_by_name error: " + str(e))
    pass

#### Release Lease

In [ ]:
delete_lease_by_name(lease_name)